## Capstone Project

### Feature Selection

#### Importing Libraries

In [1]:
%matplotlib inline

# general libraries
import re
import string
import sys
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing date libraries
import datetime as dt
import dateutil.parser as dparser

# scikit-learn libraries for preprocessing
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# scikit-learn libraries for constructing pipelines
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# scikit-learn libraries for clustering and dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.mixture import GaussianMixture

# scikit-learn libraries for evaluation
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# scikit-learn libraries for feature selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import RFECV

# scikit-learn libraries for learning
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# saving models
import pickle
from sklearn.externals import joblib

# setting pandas display options
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 10000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('precision', 5)
pd.options.mode.chained_assignment = None

#### Directory/File Structure

In [2]:
sys.version

'3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [3]:
os.getcwd()

'/Users/nate_velarde/Documents/UC_Berkeley/Courses/W210_Capstone/stroke_project/sandbox/notebooks'

In [4]:
os.chdir('../data')

In [5]:
sorted(os.listdir())

['.DS_Store',
 '273_vs_281_null_count_by_feature.csv',
 '273_vs_281_null_count_by_feature.xlsm',
 'Capstone - Complication list - complete.xlsx',
 'Capstone - STS risk factor list.xlsx',
 'Capstone_Fall_Shannon_Sept2019_request.csv',
 'capstone_STS_risk_factor_features.xlsx',
 'capstone_cleaned_data.csv',
 'capstone_data-version-2.xlsx',
 'capstone_data.xlsx',
 'capstone_data_binarized_outcome.pkl',
 'capstone_data_binarized_outcome.xlsx',
 'capstone_data_binarized_outcome_compressed.pkl',
 'capstone_data_filled_in_complication_data.xlsx',
 'capstone_data_key_variable_nulls_cleaned.pkl',
 'capstone_data_key_variable_nulls_cleaned.xlsx',
 'capstone_data_key_variable_nulls_cleaned_REF.pkl',
 'capstone_data_key_variable_nulls_cleaned_compressed.pkl',
 'pre_op_features.pkl',
 'pre_op_features_A.pkl',
 'pre_op_features_A_DREF.pkl',
 'pre_op_features_B.pkl',
 'pre_op_features_B_DREF.pkl',
 'pre_op_features_DREF.pkl',
 'pre_op_features_tree.pkl',
 'pre_op_features_tree_A.pkl',
 'pre_op_featur

#### Loading Dataset

In [6]:
data = pd.read_pickle('pre_op_features_A_DREF.pkl')

In [7]:
data.head()

,recordId,surgdt,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,predstro,strokeBin,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,length_stay,dischdt_DayOfWeek_Mon,dischdt_DayOfWeek_Tues,dischdt_DayOfWeek_Thurs,dischdt_DayOfWeek_Fri,dischdt_DayOfWeek_Sat,dischdt_DayOfWeek_Sun,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,diabctrl,dyslip,dialysis,hypertn,infendo,infendty,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,ThAoDisease,syncope,unrespstat,hitanti,TobaccoUse,cigsmoker,cigsmokercurr,chrlungd,prcvint,prcab,prvalve,chf,priorhf,medinotr,hdefd,vdaort,vdstena,vdstenm,hmo2,ivdrugab,alcohol,cvawhen,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv,CardSympTimeOfAdm_ANGINA,CardSympTimeOfAdm_STEMI,CardSympTimeOfSurg_ANGINA,CardSympTimeOfSurg_STEMI,anginalclass_SLIGHT,anginalclass_REST,classnyh_SLIGHT,classnyh_REST,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvd,cva,cvdtia,cvdpcarsurg,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_50%-79%,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_50%-79%,cvdstenlft_80-99%,cvdstenlft_100%,Arrhythmia,arrhyafib,ArrhythAFlutter_REMOTE,ArrhythAFlutter_RECENT,ArrhythAFib_PAROXYSMAL,ArrhythAFib_CONTINOUS,ArrhythAFibDur_SHORT,ArrhythAFibDur_LONG,arrhythwhen_SHORT,arrhythwhen_LONG
0,1,2011-07-01,54,180.00000,117.0,36.11111,43.0,0.9,3.8,7.2,6.5,47.0,42.0,0.014,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,5 days,0,0,0,0,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0,1.0,1.0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,1,0
1,2,2011-07-02,65,175.30000,79.4,25.83787,45.0,1.2,NaN,NaN,NaN,55.0,40.0,0.017,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,7 days,0,0,0,0,1,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,1.0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0
2,3,2011-07-04,83,162.60001,102.1,38.61754,29.0,1.2,3.3,6.2,8.6,60.0,36.0,0.045,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,8 days,0,1,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1,1.0,1.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1.0,1.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0
3,4,2011-07-05,59,160.00000,127.5,49.80469,35.0,0.9,3.5,7.4,6.4,60.0,35.0,0.013,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,4 days,0,0,0,0,1,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,1,0
4,5,2011-07-06,72,160.00000,64.0,25.00000,37.0,0.9,3.8,5.7,6.4,60.0,40.0,0.016,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,4 days,0,0,0,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,1.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,1,0


In [8]:
data.shape

(19756, 135)

#### Preprocessing

- dropping columns that you don't need in this analysis
- `recordID`, `surgdt`, `predstro`

In [9]:
drop_cols = ['recordId', 'surgdt', 'predstro']

In [10]:
data = data.drop(drop_cols, axis=1)

In [11]:
data.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,strokeBin,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,length_stay,dischdt_DayOfWeek_Mon,dischdt_DayOfWeek_Tues,dischdt_DayOfWeek_Thurs,dischdt_DayOfWeek_Fri,dischdt_DayOfWeek_Sat,dischdt_DayOfWeek_Sun,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,diabctrl,dyslip,dialysis,hypertn,infendo,infendty,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,ThAoDisease,syncope,unrespstat,hitanti,TobaccoUse,cigsmoker,cigsmokercurr,chrlungd,prcvint,prcab,prvalve,chf,priorhf,medinotr,hdefd,vdaort,vdstena,vdstenm,hmo2,ivdrugab,alcohol,cvawhen,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv,CardSympTimeOfAdm_ANGINA,CardSympTimeOfAdm_STEMI,CardSympTimeOfSurg_ANGINA,CardSympTimeOfSurg_STEMI,anginalclass_SLIGHT,anginalclass_REST,classnyh_SLIGHT,classnyh_REST,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvd,cva,cvdtia,cvdpcarsurg,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_50%-79%,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_50%-79%,cvdstenlft_80-99%,cvdstenlft_100%,Arrhythmia,arrhyafib,ArrhythAFlutter_REMOTE,ArrhythAFlutter_RECENT,ArrhythAFib_PAROXYSMAL,ArrhythAFib_CONTINOUS,ArrhythAFibDur_SHORT,ArrhythAFibDur_LONG,arrhythwhen_SHORT,arrhythwhen_LONG
0,54,180.0,117.0,36.11111,43.0,0.9,3.8,7.2,6.5,47.0,42.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,5 days,0,0,0,0,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0,1.0,1.0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,1,0
1,65,175.3,79.4,25.83787,45.0,1.2,NaN,NaN,NaN,55.0,40.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,7 days,0,0,0,0,1,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,1.0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0


In [12]:
data.shape

(19756, 132)

#### Creating `feature_matrix`

In [13]:
feature_matrix = data.copy().drop('strokeBin', axis=1)

In [14]:
feature_matrix.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,length_stay,dischdt_DayOfWeek_Mon,dischdt_DayOfWeek_Tues,dischdt_DayOfWeek_Thurs,dischdt_DayOfWeek_Fri,dischdt_DayOfWeek_Sat,dischdt_DayOfWeek_Sun,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,diabctrl,dyslip,dialysis,hypertn,infendo,infendty,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,ThAoDisease,syncope,unrespstat,hitanti,TobaccoUse,cigsmoker,cigsmokercurr,chrlungd,prcvint,prcab,prvalve,chf,priorhf,medinotr,hdefd,vdaort,vdstena,vdstenm,hmo2,ivdrugab,alcohol,cvawhen,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv,CardSympTimeOfAdm_ANGINA,CardSympTimeOfAdm_STEMI,CardSympTimeOfSurg_ANGINA,CardSympTimeOfSurg_STEMI,anginalclass_SLIGHT,anginalclass_REST,classnyh_SLIGHT,classnyh_REST,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvd,cva,cvdtia,cvdpcarsurg,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_50%-79%,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_50%-79%,cvdstenlft_80-99%,cvdstenlft_100%,Arrhythmia,arrhyafib,ArrhythAFlutter_REMOTE,ArrhythAFlutter_RECENT,ArrhythAFib_PAROXYSMAL,ArrhythAFib_CONTINOUS,ArrhythAFibDur_SHORT,ArrhythAFibDur_LONG,arrhythwhen_SHORT,arrhythwhen_LONG
0,54,180.0,117.0,36.11111,43.0,0.9,3.8,7.2,6.5,47.0,42.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,5 days,0,0,0,0,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0,1.0,1.0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,1,0
1,65,175.3,79.4,25.83787,45.0,1.2,NaN,NaN,NaN,55.0,40.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,7 days,0,0,0,0,1,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,1.0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0


In [15]:
feature_matrix.shape

(19756, 131)

### Thresholding Feature Variance
- Albon 10.1, 10.2 pages 170-172

#### Variance Thresholding Numerical Features
- need to subset numerical features - but VT will not work when feature sets contain different units (page 171) as is the case here
- replace `NaN`s with `median`
- could use `sklearn` `VarianceThreshold`, but let's manually calculate variances and standard deviations

In [16]:
numerical_features = ['age', 
                      'heightcm', 
                      'weightkg', 
                      'bmi', 
                      'hct', 
                      'creatlst', 
                      'totalbumin', 
                      'a1clvl',
                      'meldscr',
                      'hdef',
                      'pasys',
                      'length_stay'] # don't forget this is a numeric feature

In [17]:
len(numerical_features)

12

In [18]:
numerical_df = feature_matrix.copy()[numerical_features]

In [19]:
numerical_df.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,length_stay
0,54,180.0,117.0,36.11111,43.0,0.9,3.8,7.2,6.5,47.0,42.0,5 days
1,65,175.3,79.4,25.83787,45.0,1.2,NaN,NaN,NaN,55.0,40.0,7 days


In [20]:
numerical_df.shape

(19756, 12)

- replacing `NaN`s with `median`

In [21]:
numerical_df['a1clvl'].median()

5.9000001

In [22]:
numerical_df['meldscr'].median()

7.5

In [23]:
numerical_df = numerical_df.fillna(numerical_df.median())

In [24]:
numerical_df.isnull().sum()

age            0
heightcm       0
weightkg       0
bmi            0
hct            0
creatlst       0
totalbumin     0
a1clvl         0
meldscr        0
hdef           0
pasys          0
length_stay    0
dtype: int64

In [25]:
numerical_df.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,length_stay
0,54,180.0,117.0,36.11111,43.0,0.9,3.8,7.2,6.5,47.0,42.0,5 days
1,65,175.3,79.4,25.83787,45.0,1.2,3.8,5.9,7.5,55.0,40.0,7 days


- some additional pre-processing on `length_stay` 
- need to convert `timedelta` to `int`

In [26]:
numerical_df['length_stay'] = numerical_df['length_stay'].dt.days.astype('int16')

### Calculating Variance of the Numerical Features
- per Albon page 171, `Variance Thresholding` will not work when feature sets contain different units (e.g., one feature is in years while a different feature is in kg)
- then calculated the `standard deviation` of the numerical features

In [27]:
pd.DataFrame(numerical_df.var(axis=0),
             columns=['variance'])

,variance
age,122.44711
heightcm,117.83292
weightkg,417.78352
bmi,124.06981
hct,28.49372
creatlst,0.87786
totalbumin,0.27108
a1clvl,1.87235
meldscr,8.31259
hdef,150.68190


In [28]:
pd.DataFrame(numerical_df.std(axis=0),
             columns=['standard_deviation'])

,standard_deviation
age,11.06558
heightcm,10.85509
weightkg,20.43975
bmi,11.13866
hct,5.33795
creatlst,0.93694
totalbumin,0.52065
a1clvl,1.36834
meldscr,2.88316
hdef,12.27526


#### Variance Thresholding for categorical features

- how to select columns excluding a subset

In [29]:
cat_features_df = feature_matrix.copy()

In [30]:
cat_features_df = cat_features_df[cat_features_df.columns[~cat_features_df.columns.isin(numerical_features)]]

In [31]:
cat_features_df.head(2)

,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,dischdt_DayOfWeek_Mon,dischdt_DayOfWeek_Tues,dischdt_DayOfWeek_Thurs,dischdt_DayOfWeek_Fri,dischdt_DayOfWeek_Sat,dischdt_DayOfWeek_Sun,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,diabctrl,dyslip,dialysis,hypertn,infendo,infendty,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,ThAoDisease,syncope,unrespstat,hitanti,TobaccoUse,cigsmoker,cigsmokercurr,chrlungd,prcvint,prcab,prvalve,chf,priorhf,medinotr,hdefd,vdaort,vdstena,vdstenm,hmo2,ivdrugab,alcohol,cvawhen,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv,CardSympTimeOfAdm_ANGINA,CardSympTimeOfAdm_STEMI,CardSympTimeOfSurg_ANGINA,CardSympTimeOfSurg_STEMI,anginalclass_SLIGHT,anginalclass_REST,classnyh_SLIGHT,classnyh_REST,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvd,cva,cvdtia,cvdpcarsurg,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_50%-79%,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_50%-79%,cvdstenlft_80-99%,cvdstenlft_100%,Arrhythmia,arrhyafib,ArrhythAFlutter_REMOTE,ArrhythAFlutter_RECENT,ArrhythAFib_PAROXYSMAL,ArrhythAFib_CONTINOUS,ArrhythAFibDur_SHORT,ArrhythAFibDur_LONG,arrhythwhen_SHORT,arrhythwhen_LONG
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0,1.0,1.0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,1.0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0


In [32]:
cat_features_df.shape

(19756, 119)

- idea is to look to eliminate features where the vast proportion of observations is one class - say > 75%
- Albon uses the threshold of 75% of observation in one class (page 172)
- In binary features (i.e, Bernoulli random variables), variance is calculated as:
- `Var (x) = p(1 - p)`
- where `p` is the proportion of observations of class 1
- so if 75% is defined as the threshold, then want to remove features with `Variance < 0.1875)`

In [33]:
cat_variance = pd.DataFrame(cat_features_df.var(),
                            columns=['variance']).sort_values(by=['variance'],
                                                              ascending=False)

In [34]:
cat_variance.head()

,variance
status_URGENT,0.24992
anginalclass_REST,0.24980
diabetes,0.24245
vdaort,0.23348
surgdt_PartOfMonth_End,0.22194


- categorical features with `variance > 0.15`

In [35]:
cat_variance[cat_variance['variance'] >= 0.15]

,variance
status_URGENT,0.24992
anginalclass_REST,0.24980
diabetes,0.24245
vdaort,0.23348
surgdt_PartOfMonth_End,0.22194
prcvint,0.22064
surgdt_PartOfMonth_Beg,0.21877
gender,0.21058
cigsmoker,0.20434
vdstena,0.17324


In [36]:
cat_variance[cat_variance['variance'] >= 0.15].shape

(20, 1)

### Handling Highly Correlated Features

- Albon 10.3, pages 172-174
- checking to see if features are highly correlated
- creating correlation matrix

In [37]:
corr_matrix = feature_matrix.corr().abs()

In [38]:
type(corr_matrix)

pandas.core.frame.DataFrame

In [39]:
corr_matrix.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,dischdt_DayOfWeek_Mon,dischdt_DayOfWeek_Tues,dischdt_DayOfWeek_Thurs,dischdt_DayOfWeek_Fri,dischdt_DayOfWeek_Sat,dischdt_DayOfWeek_Sun,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,diabctrl,dyslip,dialysis,hypertn,infendo,infendty,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,ThAoDisease,syncope,unrespstat,hitanti,TobaccoUse,cigsmoker,cigsmokercurr,chrlungd,prcvint,prcab,prvalve,chf,priorhf,medinotr,hdefd,vdaort,vdstena,vdstenm,hmo2,ivdrugab,alcohol,cvawhen,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv,CardSympTimeOfAdm_ANGINA,CardSympTimeOfAdm_STEMI,CardSympTimeOfSurg_ANGINA,CardSympTimeOfSurg_STEMI,anginalclass_SLIGHT,anginalclass_REST,classnyh_SLIGHT,classnyh_REST,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvd,cva,cvdtia,cvdpcarsurg,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_50%-79%,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_50%-79%,cvdstenlft_80-99%,cvdstenlft_100%,Arrhythmia,arrhyafib,ArrhythAFlutter_REMOTE,ArrhythAFlutter_RECENT,ArrhythAFib_PAROXYSMAL,ArrhythAFib_CONTINOUS,ArrhythAFibDur_SHORT,ArrhythAFibDur_LONG,arrhythwhen_SHORT,arrhythwhen_LONG
age,1.00000,0.14509,0.21090,0.08379,0.15212,0.00928,0.05879,0.12195,0.08528,0.06110,0.09755,0.00976,0.01644,0.00102,0.00104,0.00349,0.01144,0.00341,0.00368,0.00294,0.01497,0.01592,0.00993,0.00426,0.00945,0.00748,0.01760,0.01046,0.00293,0.00187,0.00391,0.01026,0.04483,0.00367,0.04098,0.06269,0.08180,0.12554,0.11827,0.02001,0.02239,0.00525,0.02490,0.00542,0.06931,0.09663,0.05713,0.13403,0.10125,0.08125,0.03213,0.05562,0.00508,0.01141,0.10655,0.08643,NaN,0.05567,0.00895,0.01844,NaN,0.28893,0.31577,0.02991,0.02500,0.08903,0.03328,0.07504,0.08177,0.01182,0.00266,0.23890,0.26099,0.00442,NaN,NaN,0.06919,0.04995,NaN,NaN,0.00633,0.00567,0.05620,0.05707,NaN,NaN,NaN,NaN,0.00673,0.06428,0.06770,0.02333,0.03481,0.09803,0.07211,0.05838,0.04465,0.07756,0.08816,0.02145,0.05383,0.00078,0.01406,0.02082,0.06387,0.03578,0.01069,0.12378,0.04268,0.09039,0.10830,0.03445,0.01897,0.00894,NaN,0.03194,0.01309,NaN,0.02408,0.00014,NaN,0.14330,NaN,NaN,NaN,NaN,NaN,NaN,0.16834,0.09795
heightcm,0.14509,1.00000,0.37941,0.31306,0.22087,0.04963,0.05448,0.01277,0.06236,0.08756,0.05554,0.01340,0.00825,0.01442,0.00600,0.01101,0.00897,0.00328,0.01661,0.00651,0.00277,0.01491,0.00676,0.01193,0.00056,0.00462,0.00391,0.00947,0.01442,0.00494,0.00692,0.01672,0.00969,0.01081,0.02040,0.02235,0.64529,0.03509,0.00883,0.05379,0.00325,0.00589,0.04607,0.03263,0.02212,0.00816,0.01334,0.03271,0.01785,0.01320,0.04985,0.00919,0.03412,0.04433,0.01034,0.02078,NaN,0.00264,0.00044,0.01073,NaN,0.04253,0.03630,0.00652,0.02056,0.01060,0.00777,0.05811,0.06314,0.00396,0.00971,0.07501,0.08268,0.06763,NaN,NaN,0.08735,0.03246,NaN,NaN,0.04890,0.00558,0.01938,0.06383,NaN,NaN,NaN,NaN,0.02476,0.01103,0.04701,0.02680,0.02129,0.04164,0.04160,0.03300,0.02951,0.04912,0.06908,0.03151,0.00343,0.00323,0.00274,0.00172,0.00261,0.00595,0.00645,0.06136,0.03036,0.05316,0.03220,0.00444,0.02382,0.01867,NaN,0.01974,0.00827,NaN,0.02352,0.01586,NaN,0.01388,NaN,NaN,NaN,NaN,NaN,NaN,0.02791,0.00428
weightkg,0.21090,0.37941,1.00000,0.57497,0.15229,0.04192,0.01629,0.18166,0.05250,0.04337,0.11256,0.00543,0.00685,0.00482,0.00309,0.01007,0.00379,0.01044,0.00084,0.00330,0.00202,0.00889,0.01530,0.01670,0.00243,0.00172,0.00117,0.00051,0.00062,0.00179,0.00961,0.01140,0.0

- selecting upper triangle of correlation matrix

In [40]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),
                                  k=1).astype(np.bool))

In [41]:
type(upper)

pandas.core.frame.DataFrame

In [42]:
upper.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,dischdt_DayOfWeek_Mon,dischdt_DayOfWeek_Tues,dischdt_DayOfWeek_Thurs,dischdt_DayOfWeek_Fri,dischdt_DayOfWeek_Sat,dischdt_DayOfWeek_Sun,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,diabctrl,dyslip,dialysis,hypertn,infendo,infendty,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,ThAoDisease,syncope,unrespstat,hitanti,TobaccoUse,cigsmoker,cigsmokercurr,chrlungd,prcvint,prcab,prvalve,chf,priorhf,medinotr,hdefd,vdaort,vdstena,vdstenm,hmo2,ivdrugab,alcohol,cvawhen,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv,CardSympTimeOfAdm_ANGINA,CardSympTimeOfAdm_STEMI,CardSympTimeOfSurg_ANGINA,CardSympTimeOfSurg_STEMI,anginalclass_SLIGHT,anginalclass_REST,classnyh_SLIGHT,classnyh_REST,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvd,cva,cvdtia,cvdpcarsurg,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_50%-79%,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_50%-79%,cvdstenlft_80-99%,cvdstenlft_100%,Arrhythmia,arrhyafib,ArrhythAFlutter_REMOTE,ArrhythAFlutter_RECENT,ArrhythAFib_PAROXYSMAL,ArrhythAFib_CONTINOUS,ArrhythAFibDur_SHORT,ArrhythAFibDur_LONG,arrhythwhen_SHORT,arrhythwhen_LONG
age,NaN,0.14509,0.21090,0.08379,0.15212,0.00928,0.05879,0.12195,0.08528,0.06110,0.09755,0.00976,0.01644,0.00102,0.00104,0.00349,0.01144,0.00341,0.00368,0.00294,0.01497,0.01592,0.00993,0.00426,0.00945,0.00748,0.01760,0.01046,0.00293,0.00187,0.00391,0.01026,0.04483,0.00367,0.04098,0.06269,0.08180,0.12554,0.11827,0.02001,0.02239,0.00525,0.02490,0.00542,0.06931,0.09663,0.05713,0.13403,0.10125,0.08125,0.03213,0.05562,0.00508,0.01141,0.10655,0.08643,NaN,0.05567,0.00895,0.01844,NaN,0.28893,0.31577,0.02991,0.02500,0.08903,0.03328,0.07504,0.08177,0.01182,0.00266,0.23890,0.26099,0.00442,NaN,NaN,0.06919,0.04995,NaN,NaN,0.00633,0.00567,0.05620,0.05707,NaN,NaN,NaN,NaN,0.00673,0.06428,0.06770,0.02333,0.03481,0.09803,0.07211,0.05838,0.04465,0.07756,0.08816,0.02145,0.05383,0.00078,0.01406,0.02082,0.06387,0.03578,0.01069,0.12378,0.04268,0.09039,0.10830,0.03445,0.01897,0.00894,NaN,0.03194,0.01309,NaN,0.02408,0.00014,NaN,0.14330,NaN,NaN,NaN,NaN,NaN,NaN,0.16834,0.09795
heightcm,NaN,NaN,0.37941,0.31306,0.22087,0.04963,0.05448,0.01277,0.06236,0.08756,0.05554,0.01340,0.00825,0.01442,0.00600,0.01101,0.00897,0.00328,0.01661,0.00651,0.00277,0.01491,0.00676,0.01193,0.00056,0.00462,0.00391,0.00947,0.01442,0.00494,0.00692,0.01672,0.00969,0.01081,0.02040,0.02235,0.64529,0.03509,0.00883,0.05379,0.00325,0.00589,0.04607,0.03263,0.02212,0.00816,0.01334,0.03271,0.01785,0.01320,0.04985,0.00919,0.03412,0.04433,0.01034,0.02078,NaN,0.00264,0.00044,0.01073,NaN,0.04253,0.03630,0.00652,0.02056,0.01060,0.00777,0.05811,0.06314,0.00396,0.00971,0.07501,0.08268,0.06763,NaN,NaN,0.08735,0.03246,NaN,NaN,0.04890,0.00558,0.01938,0.06383,NaN,NaN,NaN,NaN,0.02476,0.01103,0.04701,0.02680,0.02129,0.04164,0.04160,0.03300,0.02951,0.04912,0.06908,0.03151,0.00343,0.00323,0.00274,0.00172,0.00261,0.00595,0.00645,0.06136,0.03036,0.05316,0.03220,0.00444,0.02382,0.01867,NaN,0.01974,0.00827,NaN,0.02352,0.01586,NaN,0.01388,NaN,NaN,NaN,NaN,NaN,NaN,0.02791,0.00428
weightkg,NaN,NaN,NaN,0.57497,0.15229,0.04192,0.01629,0.18166,0.05250,0.04337,0.11256,0.00543,0.00685,0.00482,0.00309,0.01007,0.00379,0.01044,0.00084,0.00330,0.00202,0.00889,0.01530,0.01670,0.00243,0.00172,0.00117,0.00051,0.00062,0.00179,0.00961,0.01140,0.01148,0.00101,0.00642,0.0

- finding index of feature columns with correlation greater than `0.95`

In [43]:
high_corr_features = [column for column in upper.columns if any(upper[column] > 0.95)]

In [44]:
high_corr_features

['cva']

In [45]:
upper[upper['cva'] > 0.95]

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,dischdt_DayOfWeek_Mon,dischdt_DayOfWeek_Tues,dischdt_DayOfWeek_Thurs,dischdt_DayOfWeek_Fri,dischdt_DayOfWeek_Sat,dischdt_DayOfWeek_Sun,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,diabctrl,dyslip,dialysis,hypertn,infendo,infendty,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,ThAoDisease,syncope,unrespstat,hitanti,TobaccoUse,cigsmoker,cigsmokercurr,chrlungd,prcvint,prcab,prvalve,chf,priorhf,medinotr,hdefd,vdaort,vdstena,vdstenm,hmo2,ivdrugab,alcohol,cvawhen,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv,CardSympTimeOfAdm_ANGINA,CardSympTimeOfAdm_STEMI,CardSympTimeOfSurg_ANGINA,CardSympTimeOfSurg_STEMI,anginalclass_SLIGHT,anginalclass_REST,classnyh_SLIGHT,classnyh_REST,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvd,cva,cvdtia,cvdpcarsurg,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_50%-79%,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_50%-79%,cvdstenlft_80-99%,cvdstenlft_100%,Arrhythmia,arrhyafib,ArrhythAFlutter_REMOTE,ArrhythAFlutter_RECENT,ArrhythAFib_PAROXYSMAL,ArrhythAFib_CONTINOUS,ArrhythAFibDur_SHORT,ArrhythAFibDur_LONG,arrhythwhen_SHORT,arrhythwhen_LONG
cvawhen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01824,0.01486,0.03244,0.03107,NaN,NaN,NaN,NaN,0.02535,0.00709,0.03748,0.03995,0.00629,0.0165,0.02847,0.00265,0.00759,0.02276,0.02193,0.00679,0.03419,0.01546,0.00563,0.00345,0.01971,0.00035,0.00238,0.6314,0.98202,0.12891,0.10898,0.04839,0.05151,0.03439,NaN,0.03672,0.04631,NaN,0.03526,0.0488,NaN,0.04434,NaN,NaN,NaN,NaN,NaN,NaN,0.0543,0.0286


- `cva` is highly correlated to `cvawhen` - not a surprise, but may consider dropping one of those features
- find out exactly what `cvawhen` is

### Removing Irrelevant Features for Classification

- Albon 10.4, pages 174-176
- for `categorical` variables, calculate a `chi-square` statistic between each `feature` and the `target` vector
- for `quantitative` variables, compute the `ANOVA F-value` between each `feature` and the `target` vector

- if we want the top 75% of features, make `k = 0.75 * number of features`

In [46]:
0.75* cat_features_df.shape[1]

89.25

- creating `target` vector

In [47]:
target = data.copy()['strokeBin']

In [48]:
target.shape

(19756,)

In [49]:
cat_features_df.shape

(19756, 119)

- selecting the top `90` features with the highest `chi-squared` statistics
- `chi-square` statistics examines the independence of two categorical vectors - that is, the statistic is the difference between the observed number of observations in each class of a categorical feature and what we would expect if that feature was independent (i.e., no relationship) with the target vector
- A `chi-square` statistic is a single number that tells you how much difference exists between your observed counts and the counts you would expect if there were no relationship at all in the population - by calculating the `chi-square` statistic between a feature and the target, we obtain a measurement of the independence between the two
- If the target is independent of the feature variable, then it is irrelevant for our purposes because it contains no information we can use for classification.
- On the other hand, if the two features are highly dependent, they likely are very informative for training models.

In [50]:
chi2_selector = SelectKBest(chi2, k=90)

In [51]:
cat_features_kbest = chi2_selector.fit_transform(cat_features_df, target)

- getting the names of the features selected

In [52]:
cat_kbest_feature_mask = chi2_selector.get_support()

In [53]:
cat_kbest_feature_mask.shape

(119,)

In [54]:
cat_kbest_features_df = pd.DataFrame(cat_features_df.columns.tolist(), columns=["Feature"])

In [55]:
cat_kbest_features_df.head()

,Feature
0,surgdt_month_Jan
1,surgdt_month_Feb
2,surgdt_month_Mar
3,surgdt_month_Apr
4,surgdt_month_May


In [56]:
cat_kbest_features_df.shape

(119, 1)

- now use the `cat_kbest_feature_mask`

In [57]:
cat_kbest_features_df = cat_kbest_features_df[cat_kbest_feature_mask == True]

- can turn this to a list to select features from main dataframe
- `cat_kbest_features` = `list(cat_kbest_features_df['Feature'].values)`
- `type(cat_kbest_features_df['Feature'].values)` yields a `numpy.ndarray`

In [58]:
cat_kbest_features_df.shape

(90, 1)

In [59]:
cat_kbest_features_df.head()

,Feature
0,surgdt_month_Jan
1,surgdt_month_Feb
2,surgdt_month_Mar
3,surgdt_month_Apr
4,surgdt_month_May


#### Let's look closely at the `chi-square` scores using the `scores_` and `pvalues_` attributes for `chi2_selector` object

In [60]:
cat_chi2_summary = pd.DataFrame(list(zip(cat_features_df.columns.tolist(),
                                         chi2_selector.scores_,
                                         chi2_selector.pvalues_)),
                                columns=['Feature', 'chi-square_statistic', 'pvalue'])

In [61]:
cat_chi2_summary.head()

,Feature,chi-square_statistic,pvalue
0,surgdt_month_Jan,1.43775,0.23050
1,surgdt_month_Feb,0.24548,0.62028
2,surgdt_month_Mar,0.94788,0.33026
3,surgdt_month_Apr,0.92780,0.33544
4,surgdt_month_May,0.38320,0.53590


- assuming a critical value of 0.05, then if `pvalue` `<=` `0.05` then we can reject the null hypothesis that the feature and target are independent -- which is what we want

#### Finding features where `pvalue` `<=` `0.05`

In [62]:
sig_cat_features = cat_chi2_summary[cat_chi2_summary['pvalue'] <= 0.05].sort_values(by=['pvalue'],
                                                                                    ascending=True)

In [63]:
sig_cat_features.shape

(32, 3)

In [64]:
sig_cat_features

,Feature,chi-square_statistic,pvalue
96,cvd,32.99659,9.23207e-09
80,classnyh_REST,24.90763,6.01438e-07
108,cvdstenlft_100%,22.14140,2.53288e-06
97,cva,19.92291,8.06284e-06
66,cvawhen,18.09469,2.10187e-05
56,chf,14.80831,1.19010e-04
83,vdinsufm_MODERATE,13.85015,1.97981e-04
44,pvd,12.90752,3.27265e-04
86,vdinsuft_MILD,12.74477,3.57006e-04
19,dischdt_DayOfWeek_Mon,10.31711,1.31802e-03


- confirming that `significant` features were in `SelectKBest`

In [65]:
in_KBest = []

for feature in sig_cat_features['Feature'].values.tolist():
    if feature in cat_kbest_features_df.values:
        in_KBest.append(1)
    else:
        in_KBest.append(0)

In [66]:
sig_cat_features['in_KBest'] = in_KBest

In [67]:
sig_cat_features

,Feature,chi-square_statistic,pvalue,in_KBest
96,cvd,32.99659,9.23207e-09,1
80,classnyh_REST,24.90763,6.01438e-07,1
108,cvdstenlft_100%,22.14140,2.53288e-06,1
97,cva,19.92291,8.06284e-06,1
66,cvawhen,18.09469,2.10187e-05,1
56,chf,14.80831,1.19010e-04,1
83,vdinsufm_MODERATE,13.85015,1.97981e-04,1
44,pvd,12.90752,3.27265e-04,1
86,vdinsuft_MILD,12.74477,3.57006e-04,1
19,dischdt_DayOfWeek_Mon,10.31711,1.31802e-03,1


- confirming

In [68]:
sig_cat_features['in_KBest'].sum() - sig_cat_features.shape[0]

0

#### Now let's use the same coding pattern with the numerical features, except using `ANOVA F-value` between each numerical feature and target vector

In [69]:
numerical_df.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,length_stay
0,54,180.00000,117.0,36.11111,43.0,0.9,3.8,7.2,6.5,47.0,42.0,5
1,65,175.30000,79.4,25.83787,45.0,1.2,3.8,5.9,7.5,55.0,40.0,7
2,83,162.60001,102.1,38.61754,29.0,1.2,3.3,6.2,8.6,60.0,36.0,8
3,59,160.00000,127.5,49.80469,35.0,0.9,3.5,7.4,6.4,60.0,35.0,4
4,72,160.00000,64.0,25.00000,37.0,0.9,3.8,5.7,6.4,60.0,40.0,4


In [70]:
numerical_df.shape

(19756, 12)

In [71]:
target.shape

(19756,)

- instantiating a `SelectKBest` object

In [72]:
fvalue_selector = SelectKBest(f_classif, k=6)

In [73]:
num_features_kbest = fvalue_selector.fit_transform(numerical_df, target)

- getting the names of the features selected

In [74]:
num_kbest_feature_mask = fvalue_selector.get_support()

In [75]:
num_kbest_feature_mask[0:5]

array([ True,  True,  True, False,  True])

In [76]:
num_kbest_feature_mask.shape

(12,)

- creating a `dataframe` of the numerical feature names

In [77]:
num_kbest_features_df = pd.DataFrame(numerical_df.columns.tolist(), columns=["Feature"])

- now use the `cat_kbest_feature_mask`

In [78]:
num_kbest_features_df = num_kbest_features_df[num_kbest_feature_mask == True]

- yields the `6` best features

In [79]:
num_kbest_features_df

,Feature
0,age
1,heightcm
2,weightkg
4,hct
10,pasys
11,length_stay


#### Let's look closely at the `F-values` scores using the `scores_` and `pvalues_` attributes for `fvalue_selector` object

In [80]:
num_fvalue_summary = pd.DataFrame(list(zip(numerical_df.columns.tolist(),
                                           fvalue_selector.scores_,
                                           fvalue_selector.pvalues_)),
                                  columns=['Feature', 'F_value_statistic', 'pvalue'])

In [81]:
num_fvalue_summary

,Feature,F_value_statistic,pvalue
0,age,44.78792,2.25462e-11
1,heightcm,18.85836,1.41493e-05
2,weightkg,25.48905,4.48857e-07
3,bmi,5.09254,2.40399e-02
4,hct,19.45003,1.03812e-05
5,creatlst,3.96882,4.63640e-02
6,totalbumin,16.70362,4.38702e-05
7,a1clvl,2.16525,1.41178e-01
8,meldscr,5.32159,2.10731e-02
9,hdef,10.02233,1.54889e-03


#### Finding features where `pvalue` `<=` `0.05`

In [82]:
sig_num_features = num_fvalue_summary[num_fvalue_summary['pvalue'] <= 0.05].sort_values(by=['pvalue'],
                                                                                        ascending=True)

In [83]:
sig_num_features

,Feature,F_value_statistic,pvalue
11,length_stay,432.27065,5.41580e-95
0,age,44.78792,2.25462e-11
2,weightkg,25.48905,4.48857e-07
4,hct,19.45003,1.03812e-05
1,heightcm,18.85836,1.41493e-05
10,pasys,17.37037,3.08892e-05
6,totalbumin,16.70362,4.38702e-05
9,hdef,10.02233,1.54889e-03
8,meldscr,5.32159,2.10731e-02
3,bmi,5.09254,2.40399e-02


In [84]:
sig_num_features.shape

(11, 3)

- all `numerical` features are significant